In [20]:
import pandas as pd 

### Data
* mRNA_df (1408, 19194) | OmicsExpressionProteinCodingGenesTPMLogp1.csv
    * Depmap ID
* gene_dependency_df (1095, 17932) | CRISPRGeneDependency.csv
    * ModelID
* cells_df (103, 6) | Depmap Melanoma Cell Lines.csv
    * Depmap Id

In [88]:
# mRNA
mRNA_df = pd.read_csv(r'C:\Users\commo\OneDrive - University of Virginia\School\STEM\BME\BME 4700 - Systems Bioengineering\CCLE Chromatin Analysis\Raw Data\OmicsExpressionProteinCodingGenesTPMLogp1.csv')
print(mRNA_df.shape)


(1408, 19194)


In [8]:
# Gene Dependency
gene_dependency_df = pd.read_csv(r'C:\Users\commo\OneDrive - University of Virginia\School\STEM\BME\BME 4700 - Systems Bioengineering\CCLE Chromatin Analysis\Raw Data\CRISPRGeneDependency.csv')
print(gene_dependency_df.shape)


(1095, 17932)


In [12]:
# DepMap Melanome Cell Lines
cells_df = pd.read_csv(r'C:\Users\commo\OneDrive - University of Virginia\School\STEM\BME\BME 4700 - Systems Bioengineering\CCLE Chromatin Analysis\Raw Data\Depmap Melanoma Cell Lines.csv')
cells_df.drop(columns=['Unnamed: 0'], inplace=True)
print(cells_df.shape)


(103, 6)


### ID Matching

* 60 Cell Lines match in both, limited by gene dependency data

In [19]:
# Matches mRNA_df and gene_dependency_df
matches_df = pd.merge(mRNA_df, gene_dependency_df, left_on='Depmap ID', right_on = 'ModelID', how='inner')
print(matches_df.shape)

(1001, 37126)


In [16]:
# Matches matches_df and cells_df
cell_matches_df = pd.merge(matches_df, cells_df, left_on='Depmap ID', right_on = 'Depmap Id', how='inner')
print(cell_matches_df.shape)

(60, 37132)


### Save Cell Matches

#### Preprocessing

In [37]:
# Index into mRNA by list(cell_matches_df['Depmap ID'])
depmap_mRNA_df = mRNA_df[mRNA_df['Depmap ID'].isin(cell_matches_df['Depmap ID'])]
print(depmap_mRNA_df.shape)

# Index into gene_dependency by list(cell_matches_df['Depmap ID'])
depmap_gene_dependency_df = gene_dependency_df[gene_dependency_df['ModelID'].isin(cell_matches_df['Depmap ID'])]
print(depmap_gene_dependency_df.shape)

(60, 19194)
(60, 17932)


In [39]:
# Remove (xxxx) from columns
depmap_gene_dependency_df.columns = depmap_gene_dependency_df.columns.str.replace(r" \(.*\)","")
depmap_mRNA_df.columns = depmap_mRNA_df.columns.str.replace(r" \(.*\)","")


C:\Users\commo\AppData\Local\Temp\ipykernel_22936\881064277.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  depmap_gene_dependency_df.columns = depmap_gene_dependency_df.columns.str.replace(r" \(.*\)","")
C:\Users\commo\AppData\Local\Temp\ipykernel_22936\881064277.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  depmap_mRNA_df.columns = depmap_mRNA_df.columns.str.replace(r" \(.*\)","")


In [69]:
# Remap Depmap ID to Cell Line Name with depmap_cells_df as a dictionary
id_cell_dict = dict(zip(cells_df['Depmap Id'], cells_df['Cell Line']))

# Remap Depmap ID to Cell Line Name with depmap_cells_df as a dictionary
depmap_gene_dependency_df['ModelID'] = depmap_gene_dependency_df['ModelID'].map(id_cell_dict)
depmap_mRNA_df['Depmap ID'] = depmap_mRNA_df['Depmap ID'].map(id_cell_dict)

C:\Users\commo\AppData\Local\Temp\ipykernel_22936\1094932552.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depmap_gene_dependency_df['ModelID'] = depmap_gene_dependency_df['ModelID'].map(id_cell_dict)


In [62]:
# Index to retrieve cell metadata
depmap_cells_df = cell_matches_df.iloc[:, len(cell_matches_df.columns)-5:]

In [72]:
# Sort by Cell Line Name
depmap_gene_dependency_df.sort_values(by=['ModelID'], inplace=True)
depmap_mRNA_df.sort_values(by=['Depmap ID'], inplace=True)
depmap_cells_df.sort_values(by=['Cell Line'], inplace=True)

# Rename Columns
depmap_gene_dependency_df.rename(columns={'ModelID':'Cell Line'}, inplace=True)
depmap_mRNA_df.rename(columns={'Depmap ID':'Cell Line'}, inplace=True)


C:\Users\commo\AppData\Local\Temp\ipykernel_22936\1118756077.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depmap_gene_dependency_df.sort_values(by=['ModelID'], inplace=True)
C:\Users\commo\AppData\Local\Temp\ipykernel_22936\1118756077.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depmap_gene_dependency_df.rename(columns={'ModelID':'Cell Line'}, inplace=True)


In [89]:
mRNA_df.columns = depmap_mRNA_df.columns.str.replace(r" \(.*\)","")


C:\Users\commo\AppData\Local\Temp\ipykernel_22936\99317108.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  mRNA_df.columns = depmap_mRNA_df.columns.str.replace(r" \(.*\)","")


In [91]:
mRNA_df[1:].to_csv('Depmap Pancancer mRNA.csv')

#### Saves

In [83]:
# Save to CSV
depmap_cells_df.to_csv('Depmap Melanoma Cells.csv')
depmap_gene_dependency_df.to_csv('Depmap Melanoma Gene Dependency.csv')
depmap_mRNA_df.to_csv('Depmap Melanoma mRNA.csv')